# DPAA MIA/KIA Graph Viz
Author: John Bonfardeci<br />
Last modified: 2021-05-12

In [1]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
!conda update -n base -c defaults conda -y

Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::tifffile==2021.1.14=pyhd3eb1b0_1
  - defaults/linux-64::numpy-base==1.19.2=py37hfa32c7d_0
  - conda-forge/linux-64::numexpr==2.7.3=py37hdc94413_0
  - defaults/linux-64::secretstorage==3.3.1=py37h06a4308_0
  - defaults/linux-64::bokeh==2.2.3=py37_0
  - defaults/linux-64::anaconda-client==1.7.2=py37_0
  - defaults/linux-64::bottleneck==1.3.2=py37heb32a55_1
  - defaults/linux-64::imagecodecs==2021.1.11=py37h581e88b_1
  - defaults/linux-64::keyring==22.0.1=py37h06a4308_0
  - defaults/noarch::dask==2021.2.0=pyhd3eb1b0_0
  - defaults/linux-64::_anaconda_depends==2020.07=py37_0
  - defaults/linux-64::mkl_fft==1.3.0=py37h54f3939_0
  - defaults/linux-64::scikit-learn==0.23.2=py37h0573a6f_0
  - defaults/linux-64::spyder==4.2.1=py37h06a4308_1
  - defaults/linux-64::harfbuzz==2.4.0=hca77d97_1
  - defaults/linux-64::pywavelets==

In [8]:
!conda install pyodbc -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.4
  latest version: 4.10.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/JupyterSystemEnv

  added / updated specs:
    - pyodbc


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    unixodbc-2.3.9             |       hb166930_0         293 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         293 KB

The following NEW packages will be INSTALLED:

  pyodbc             conda-forge/linux-64::pyodbc-4.0.30-py36hc4f0c31_1
  unixodbc           conda-forge/linux-64::unixodbc-2.3.9-hb166930_0



unixodbc-2.3.9       | 293 KB    | ##################################### | 100% 
Preparing transaction: done
Verify

In [2]:
%graph_notebook_version

2.1.2


In [3]:
%graph_notebook_config

{
  "host": "database-1.cluster-cbqe6d3hp7zw.us-east-1.neptune.amazonaws.com",
  "port": 8182,
  "auth_mode": "DEFAULT",
  "load_from_s3_arn": "",
  "ssl": true,
  "aws_region": "us-east-1",
  "sparql": {
    "path": "sparql"
  }
}


In [4]:
%status

{'status': 'healthy',
 'startTime': 'Fri May 14 02:03:27 UTC 2021',
 'dbEngineVersion': '1.0.4.1.R4',
 'role': 'writer',
 'gremlin': {'version': 'tinkerpop-3.4.8'},
 'sparql': {'version': 'sparql-1.1'},
 'labMode': {'NeptuneML': 'disabled',
  'ObjectIndex': 'disabled',
  'DFEQueryEngine': 'disabled',
  'ReadWriteConflictDetection': 'enabled'}}

### List Total Number of People in Database

In [5]:
%%gremlin
g.V().hasLabel('person').groupCount().by(label)

### Query visualization hints

You can specify query visualization hints using either `-p` or `--path-pattern` after the `%%gremin` cell "magic". The syntax in general is:

`%%gremlin -p | --path-pattern <comma separated hints>`

 The names of the hints reflect the Gremlin steps most commonly used when traversing between vertices and behave accordingly. The hints used should match the corresponding Gremlin steps used in the query. The hints used can be any combination of those shown in the list below, separated by commas. The list must not contain any spaces between the commas.

- v
- inv
- outv
- e
- ine
- oute



We can provide visualization hints for the query shown earlier as follows:

`%%gremlin -p v,oute,inv`  
`g.V().hasLabel('airport').outE().inV().path().by('code').by('dist').limit(5)`

You will find that query in the cell below. Try running it with and without the hints present and observe the differences to the visualization by selecting the `Graph` tab. Notice that without the hint the visualizer is unable to determine if the `dist` property relates to a vertex or an edge and therefore defaults to using a vertex.

### Adjusting the visualization layout and other settings
You can further adjust many of the visualization settings using the two commands

- `%graph_notebook_vis_options`
- `%%graph_notebook_vis_options`

There are examples of how to use these settings and links to additional documentation at the bottom of this notebook.

In [6]:
%%gremlin -p v,oute,inv
g.V().has('person', 'person_id', '0b03d5e1812c4ca9').outE().inV().path().limit(2500)

## List all Vertices

In [7]:
%%gremlin
g.V().groupCount().by(label).unfold()

## List all Edges

In [8]:
%%gremlin
g.E().groupCount().by(label).unfold()

## Count of identified = 1/0

In [9]:
%%gremlin
g.V().hasLabel('identified').group().by('value').by(in().count()).unfold()

### Relationships for First Ten Soldiers in DB

In [10]:
%%gremlin
g.V().hasLabel('person').limit(10)
    .outE('served_in', 'served_under', 'has_conflict_status_code', 'has_identified_status')
    .inV().values('value')
    .path().limit(1000)

ERROR:asyncio:Task was destroyed but it is pending!
task: <Task pending coro=<WebSocketProtocol13._receive_frame_loop() done, defined at /home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.6/site-packages/tornado/websocket.py:1113> wait_for=<Future pending cb=[Task._wakeup()]> cb=[IOLoop.add_future.<locals>.<lambda>() at /home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.6/site-packages/tornado/ioloop.py:688]>


### Traverse Vietnam War by Person and Identified Status

In [11]:
%%gremlin
g.V()
    .has('conflict', 'value', 'VIETNAM WAR')
    .inE().outV().hasLabel('person')
    .outE().inV().hasLabel('identified').values('value')
    .path().limit(100)

ERROR:asyncio:Task was destroyed but it is pending!
task: <Task pending coro=<WebSocketProtocol13._receive_frame_loop() running at /home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.6/site-packages/tornado/websocket.py:1116> wait_for=<Future pending cb=[Task._wakeup()]> cb=[IOLoop.add_future.<locals>.<lambda>() at /home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.6/site-packages/tornado/ioloop.py:688]>
ERROR:asyncio:Task was destroyed but it is pending!
task: <Task pending coro=<WebSocketProtocol13._receive_frame_loop() running at /home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.6/site-packages/tornado/websocket.py:1116> wait_for=<Future pending cb=[Task._wakeup()]> cb=[IOLoop.add_future.<locals>.<lambda>() at /home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.6/site-packages/tornado/ioloop.py:688]>
ERROR:asyncio:Task was destroyed but it is pending!
task: <Task pending coro=<WebSocketProtocol13._receive_frame_loop() running at /home/ec2-user/anaconda

In [16]:
%%gremlin
g.V()
    .has('conflict', 'value', 'VIETNAM WAR')
    .inE().outV().hasLabel('person')
    .outE().inV().hasLabel('identified').values('value')
    .path().limit(100)

### Korean War

In [7]:
%%gremlin
g.V()
    .has('conflict', 'value', 'KOREAN WAR')
    .inE().outV().hasLabel('person').limit(25)
    .out()
    .path().limit(1000)

ERROR:asyncio:Task was destroyed but it is pending!
task: <Task pending coro=<WebSocketProtocol13._receive_frame_loop() running at /home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.6/site-packages/tornado/websocket.py:1116> wait_for=<Future pending cb=[Task._wakeup()]> cb=[IOLoop.add_future.<locals>.<lambda>() at /home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.6/site-packages/tornado/ioloop.py:688]>
